# Piezo power supply
In order to supply maximum power for the piezo along the resonance point, we need
- A square-wave generator (stm32)
- A low-pass filter (optional)
- A FET
- A FET driver (to drive 2.5 amps to the piezo-element at resonance)

## Solution 1 
- Using a bldc driver board / buy some random inverter board, and drive the piezo-electric using a voltage source.
	- Make sure to check the voltage wave coming from the inverter, to check if a band-pass filter is needed.
- Switching frequency:
	- Dead time is 520 ns, so 1 us is about 1 MHz which is definitely too slow for our purpose.
- Most of the regular stuff doesn't switch at MHz frequencies

### Conclusion
- Too slow

## Solution 2: single fet drive
- Use a UCC2751x OR a UCC27611 as a FET driver
	- the UCC27611 
		- has better rise / fall times and propagation delays
		- It also has built-in circuitry for level shifting
		- Is really expensive
	- the UCC27611
		- Doesn't have level shifting (but we also don't need it)
	- fall + rise + dead times here are (td1+tr+td2+tf) * 10 = 10+20+20 = 50 ns, our switching period needs to be (1/2MHz) / 2 = 250 ns [since each side of the piezo-electric needs switch every 250 ns]. 250 ns / 50 = 5 times the rise + fall time. 
		- Given that we really need a sine-wave, and we want to filter out the higher frequency components anyways, it's acceptable.

### Conclusion
- We need a bidirection / AC drive

## Solution 3
- Use a 4 x 1EDN8550BXTSA1 to form a full-bridge AC-drive
	- Add bootstrap circuitry for high-side supply.
	- Low side supply simply connected to PWM output.
- Add 4 x NDS355AN as FET's

### Can we use a single PWM signal for both drivers without creatinga short?
No we can't, otherwise you need 
- Dead time
- Separation between the 2 comparators

### Conclusion
- Use another model, this model seems to not have any decent spice models available, only pspice.

## Solution 4
- 4 x NDS355AN as FET's
	- Can handle a max of 30 Volts
	- Can handle 
		- max 10 amps of pulsed current
		- max 1.7 amps of continuous current
	- Vgs_max = 
- The UCC27282
	- requires addition buck-converter / LDO circuitry for supplying lower voltages
	- Vgs_max= 20 V
		- Best keep it between 5 and 10 volts
	
- Start with a single transceiver and a single receiver channel to bring down cost, once it works increase the n of channels

## Solution 5
- Use the SI7454FDP-T1-RE3, it is more expensive but it can handle
	- Voltages Vds = 100 V max (which is also good with the bootstrap voltage)
	- Id: 23.5 amps
	- This will allow us to drive the fet with voltages up to 50 volts, with current peaks up to 30 amps if required
	- Spice model is for Si7454DP instead of Si7454FDP
		- Differences: FDP takes ore drain current
		- FDP is faster, has much less gate charge
		- So whatever we simulate with this model, will be better in real life

In [9]:

#! Voltage drop over gate driver
Q_gate = 22e-9 # 5 nC maximum
Vdd = 12 # Supply for driver circuit (keep it between 10 and 12 volts) -> This makes sure switching times are at their lowest
Vdh = 1 # Internal UCC27282 diode forward voltage drop
# Keep vgs above 4.5 volts to be on the safe side
Vgs_min = 10 # Lower side gate-source voltage for the SI7454FDP-T1-RE3 to conduct
dvhb_max = Vdd-Vdh-(Vgs_min) # Maximum voltage drop allowed across the capacitor
print(f"Max voltage drop across the capacitor: {dvhb_max}")

#! Charge needed per switching cycle
I_hb = 0.5e-3 # Quiescent current from Hb to Vss
I_hbs = 50e-6 # Leakage current from Hb to Vss
D_max = 0.5 # Duty cycle
f_sw = 2e6 # 2 MHz switching frequency
Q_total = Q_gate + I_hbs * (D_max / f_sw) + (I_hb / f_sw)
C_boot = Q_total / dvhb_max # Required bootstrap capacitance
C_boot_safe = C_boot * 10
print(f"Bootstrap capacitor requirement: {C_boot}, safe: {C_boot_safe}")
# Pick bootstrap cap around 230 nF

Max voltage drop across the capacitor: 1
Bootstrap capacitor requirement: 2.2262499999999998e-08, safe: 2.22625e-07


### Bootstrap capacitor decision
- For now choose a bootstrap capacitor of about 20 nF to be on the safe side, especially when going to lower frequencies it might be necessary.


In [15]:

#! External gate resistor selection
## HIGH SIDE RESISTOR
V_dh = 1 # Forward voltage drop bootstrap diode

#* Equivalent gate resistance
t_don = 15e-9 # Delay on-time
C_iss = 195e-12 # Input capacitance
R_gfet = 1 # Estimate of the gate resistance
R_gate = 11 # External gate resistance connected
R_loh = R_lol = R_hol = R_hoh = (0.3 / 3) # Gate driver internal high-side pull-up resistor (pull-up voltage / pull-up current)
I_o = (Vdd - V_dh) / (R_hoh + R_gate + R_gfet)
print(f"Pull-up / Down currents for high and low-side: {round(I_o, 2)} A for external resistance {R_gate}")

Pull-up / Down currents for high and low-side: 0.91 A for external resistance 11


### Gate resistor decision
- For now simply put a 10-11 ohm-resistor for current limiting. The peak pull-up / pull-down currents are about 3 amps.

# Sources:
- https://www.ti.com/lit/an/snoa390b/snoa390b.pdf